In [7]:
# !pip install --upgrade pip 
# !pip install selenium
# !pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1279 sha256=c39669ba50a9c8d6444b5147dac700ca5f2b7c5858f84e6ec0a5657d9b5cddf5
  Stored in directory: c:\users\107lab\appdata\local\pip\cache\wheels\19\f5\6d\a97dd4f22376d4472d5f4c76c7646876052ff3166b3cf71050
Successfully built bs4


In [1]:
%matplotlib inline
import numpy as np
from selenium import webdriver
# from selenium.webdriver.chrome.options import Options # 創建chrome參數對象
import time
import datetime
from bs4 import BeautifulSoup
import sys
import random
# import matplotlib.pylab as plt

In [10]:
# driver路徑位置
driver = webdriver.Chrome('D:/自己研究資料/NLP/chromedriver')

# 搜尋網址
sourceurl ='https://twitter.com/search?q=CNN&src=typed_query&f=live'
driver.get(sourceurl)

In [11]:
# 測試 確認剛load網頁時 有沒有讀到資料了
soap = driver.page_source
soap = BeautifulSoup(soap,from_encoding="UTF-8")
alldata = soap.find_all("div",class_ = "css-901oao r-18jsvk2 r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0")
alldate = soap.find_all("a",class_ = "css-4rbku5 css-18t94o4 css-901oao r-14j79pv r-1loqt21 r-1q142lx r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-3s2u2q r-qvutc0")
print(len(alldata),len(alldate))


10 10


In [12]:
def get_data():
    time.sleep(5)
    # 每加載資料都要重新 load source 的 data，不然內容不會改變
    soap = driver.page_source
    soap = BeautifulSoup(soap,from_encoding="UTF-8")
    # all_data為tweets內容,all_date為年月日,datetime為tag裡的日期
    all_data = soap.find_all("div",class_ = "css-901oao r-18jsvk2 r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0")
    all_date = soap.find_all("a",class_ = "css-4rbku5 css-18t94o4 css-901oao r-14j79pv r-1loqt21 r-1q142lx r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-3s2u2q r-qvutc0")
    datetime = soap.find_all("time")
    # 存後的height有多高，滾到底時防止一直loop最下方的幾筆資料
    scroll_height = driver.execute_script("return document.documentElement.scrollHeight")
    height.append(scroll_height)
    print(len(all_data))
    return (all_data , all_date,scroll_height,height,datetime)

In [13]:
content = []
date = []
height = [0]
i = 0
all_data , all_date , scroll_height , height ,datetime = get_data()
# 當一樣長度時
while i < len(all_data):
    time.sleep(5)
    print(str(i+1) + "/" + str(len(all_data)) + "  height: " +str(scroll_height))
    datetime[i] = datetime[i].get('datetime').replace('.000Z','').replace('T',' ').replace('-','/') 
    print(datetime[i],all_data[i].text)
#     print(datetime[i].get('datetime').replace('.000Z','').replace('T',' ').replace('-','/'),all_data[i].text)
#     print(all_date[i].text,all_data[i].text)
    content.append(all_data[i].text)
    date.append(datetime[i])
    i += 1
    if i >= (len(all_data)):
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);') # 讓頁面滾動動
        i = 0
        all_data , all_date , scroll_height , height ,datetime = get_data()
        
        if height[-1] == height[-2]:
            all_data , all_date , scroll_height , height ,datetime = get_data()
            print('資料到底，可能重複了')
            break

10
1/10  height: 5072
2022/01/13 02:39:29 CNN's getting crushed in the ratings.
2/10  height: 5072
2022/01/13 02:39:29 Hard pass
3/10  height: 5072
2022/01/13 02:39:29 Thank you. It’s always great to be with @thelauracoates on @CNN . She brings the .
4/10  height: 5072
2022/01/13 02:39:24 Also Covid started out as a sexually transmitted disease in Wuhan’s homosexual population. It was not a respiratory virus at first. It evoked into a respiratory virus rapidly due to the # of men transmitting it back and forth to each other. The first to go to the hospital
5/10  height: 5072
2022/01/13 02:39:21 Isn't a member of the Saudi "royal" family a partial owner of Fox "news?" The US mainstream media only reports what they want you to know, that's not all the innocent lives being lost in besieged Yemen or Palestine. #CNN #FOXNEWS #bbcamerica
6/10  height: 5072
2022/01/13 02:39:19 Fox News shows make America less safe. Fox isn’t real news, it’s entertainment to radicalize Americans to join Terror

KeyboardInterrupt: 

In [14]:
import pandas as pd
df = pd.DataFrame({'created_at':date,'text':content})

df.to_csv('test.csv',encoding='utf-8-sig')